In [1]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize
from tqdm import tqdm

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.array([[5.09229699e-09,  1.00000000e+00,  5.40467264e-10], [ 1.00000000e+00, -4.00000000e+00,  1.00000000e+00], [-2.81036184e-09,  1.00000000e+00,  1.97387560e-08]]) #initial kernel


# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Objective function to minimize
def objective_function(kernel_params, training_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for image in training_images:
        denoised_image = image.copy()
        for _ in range(20):
            denoised_image = denoise_image(denoised_image, kernel_padded)
        error += np.mean((denoised_image - image) ** 2)  # Example error metric (MSE)
    return error

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u**2)
    p = dt * phi 
    return image + p

# Load the training images from a directory
training_dir = 'mult_train'
training_images = []
for filename in os.listdir(training_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image = cv2.imread(os.path.join(training_dir, filename), 0)
        image = image.astype(np.float32) / 255.0
        training_images.append(image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=len(training_images), desc='Training Kernel') as pbar:
    def update_progress(_):
        pbar.update(1)

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(training_images,),
                                callback=update_progress)

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test_image.png', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = test_image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, trained_kernel_padded)

cv2.imwrite('denoised_image.jpg', denoised_image * 255.0)


Training Kernel:   1%|▍                                                            | 3/400 [10:21<22:51:19, 207.25s/it]


True

In [2]:
trained_kernel

array([[ 2.41205201e-03,  1.00000000e+00,  6.61658879e-04],
       [ 1.00000000e+00, -4.00000000e+00,  1.00000000e+00],
       [ 1.40655918e-03,  1.00000000e+00,  8.08768160e-03]])